In [1]:
!pip install -U spacy -q

In [1]:
import spacy

In [3]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("id") # load a new spacy model
db = DocBin() # create a DocBin object

In [4]:
import json
f = open('annotations (2).json')
TRAIN_DATA = json.load(f)

In [5]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 80/80 [00:00<00:00, 4458.23it/s]


In [6]:
! python -m spacy init config config.cfg --lang id --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only), install the
spacy-transformers package and re-run this command. The config generated now
does not use transformers.
ℹ Generated config template specific for your use case
- Language: id
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-08 16:21:34.844709: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-08 16:21:34.845061: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     49.00    0.00    0.00    0.00    0.00
 18     200         55.49   1374.17   95.42   94.81   96.05    0.95
 41     400         37.98    127.80  100.00  100.00  100.00    1.00
 70     600          0.64      0.70  100.00  100.00  100.00    1.00
105     800          0.00      0.00  100.00  100.00  100.00    1.00
149    1000          0.00      0.00  100.00  100.00  100.00    1.00
199    1200          0.00      0.00  100.00  100.00  100.00    1.00
266    1400          0.00      0.00  100.00  100.00  100.00    1.00
341    1600          0.00      0.00 

2022-05-08 16:21:54.041534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-08 16:21:54.041975: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-08 16:22:01,096] [INFO] Set up nlp object from config
[2022-05-08 16:22:01,104] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-08 16:22:01,107] [INFO] Created vocabulary
[2022-05-08 16:22:01,142] [INFO] Finished initializing nlp object
[2022-05-08 16:22:01,273] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [8]:
ner_model = spacy.load('model-best')

In [10]:
doc = ner_model('''
"intent": "qualification-DS",
"apa kompetensi yang dibutuhkan role data scientist?",
"apa kompetensi yang dibutuhkan posisi data scientist?",
"kompetensi buat scientist apa aja?",
"data scientist harus bisa apa?",
"lulusan non teknik bisa daftar jadi data scientist?",
"kalau ga bisa IT bisa jadi data scientist?",
"kalau belum punya pengalaman bisa jadi data scientist?",
"kualifikasi data scientist gimana?",
"daftar jadi data scientist harus bisa apa aja?",
"kalau jurusan tidak tertulis apa bisa jadi data scientist?",
"bagaimana kualifikasi data scientist yang dicari?",
"kalau ga bisa data science boleh daftar data scientist?",
"ga bisa coding boleh daftar data science?",
"ga bisa coding boleh daftar data scientist?",
"ga ngerti data science tapi mau daftar",
"belum punya pengalaman mau daftar data scientist",
"pengalaman lebih dari tiga tahun mau lamar data scientist"
"Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"
''')

In [11]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [13]:
doc = ner_model('''
data scientist itu apa, data engineer, analis, data analyst, data analist, data science, DS, DA, DE science, analyst, engine, engineer
''')

In [2]:
doc.ents

NameError: name 'doc' is not defined

In [14]:
for ent in doc.ents:
  print(ent.text, ent.label_)

data scientist SCIENTIST
data engineer ENGINEER
analis ANALIS
data analyst ANALIS
data analist SCIENTIST
data science SCIENTIST
DS SCIENTIST
DA ANALIS
engineer ENGINEER


In [15]:
for ent in doc.ents:
  print(ent.label_)

SCIENTIST
ENGINEER
ANALIS
ANALIS
SCIENTIST
SCIENTIST
SCIENTIST
ANALIS
ENGINEER


In [16]:
cek = ner_model('''
Data scientis ngapain aja
''')

In [17]:
for ent in cek.ents:
    print(ent.label_)

ANALIS


In [18]:
for ent in cek.ents:
    print(ent.label_)
    if ent.label_ == "ANALIS":
        print('yes')

ANALIS
yes
